# MTSA


## install


In [3]:
!pip3 install shap



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
!pip3 install mtsa


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [28]:
!git clone https://github.com/diegompin/mtsa.git


fatal: destination path 'mtsa' already exists and is not an empty directory.


In [29]:
!git checkout feature/v0.0.13-c3-gabrieltorres


M	examples/MTSA_+_SHAP.ipynb
M	examples/mtsa
Already on 'feature/v0.0.13-c3-gabrieltorres'
Your branch is up to date with 'origin/feature/v0.0.13-c3-gabrieltorres'.


In [30]:
import os
import shap

import numpy as np

from sklearn.mixture import GaussianMixture
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

from mtsa import files_train_test_split
from mtsa import calculate_aucroc

from mtsa.utils import Wav2Array
from mtsa.features import Array2Mfcc
from mtsa.features import MagnitudeMeanFeatureMfcc
from mtsa.features import MagnitudeStdFeatureMfcc
from mtsa.features import CorrelationFeatureMfcc

from mtsa.models import MFCCMix
from mtsa.models import MFCCLocalOutlierFactor

ImportError: cannot import name 'MFCCLocalOutlierFactor' from 'mtsa.models' (/usr/local/lib/python3.8/dist-packages/mtsa/models/__init__.py)

## import dados


In [7]:
path_input = "/data/gabriel/valve/id_02"

In [8]:
X_train, X_test, y_train, y_test = files_train_test_split(path_input)

print("Dimensões dos dados de treino: ", X_train.shape)
print("Dimensões dos dados de teste: ",X_test.shape)
print("Dimensões dos rótulos de treino: ",y_train.shape)
print("Dimensões dos rótulos de teste: ",y_test.shape)


Dimensões dos dados de treino:  (588,)
Dimensões dos dados de teste:  (240,)
Dimensões dos rótulos de treino:  (588,)
Dimensões dos rótulos de teste:  (240,)


## Pré-processamento dos sinais:

In [9]:
# Instanciando os objetos
wav2array = Wav2Array(16000)
array2mfcc = Array2Mfcc(16000)
M = MagnitudeMeanFeatureMfcc()
S = MagnitudeStdFeatureMfcc()
C = CorrelationFeatureMfcc()

In [10]:
# Convertendo os sinais para array
array_X_train = wav2array.transform(X_train)
array_X_test = wav2array.transform(X_test)

# Convertendo os arrays para mfcc
mfcc_X_train = array2mfcc.transform(array_X_train)
mfcc_X_test = array2mfcc.transform(array_X_test)

# Convertendo mfcc para as features
M_X_train = M.transform(mfcc_X_train)
S_X_train = S.transform(mfcc_X_train)
C_X_train = C.transform(mfcc_X_train)
M_X_test = M.transform(mfcc_X_test)
S_X_test = S.transform(mfcc_X_test)
C_X_test = C.transform(mfcc_X_test)

# Concatenando as features
concat_X_train = np.concatenate((M_X_train, S_X_train, C_X_train), axis=1)
print("Dimensões das features de treino: ", concat_X_train.shape)

concat_X_test = np.concatenate((M_X_test, S_X_test, C_X_test), axis=1)
print("Dimensões das features de teste: ", concat_X_test.shape)

Dimensões das features de treino:  (588, 230)
Dimensões das features de teste:  (240, 230)


In [11]:
print("Rótulos de treino: ", y_train)
print("Rótulos de teste: ",y_test)

Rótulos de treino:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 

## Treinar modelo MFCCMix:

In [12]:
mfccmix = MFCCMix()
mfccmix.fit(X_train, y_train)

print("MFCCMIX")
print("\nTrain: ")
print("Modelo: ", mfccmix.predict(X_train))
print("Real: ", y_train)
print("Score samples: ", mfccmix.score_samples(X_train))

print("\nTest: ")
print("Modelo: ", mfccmix.predict(X_test))
print("Real: ", y_test)
print("Score samples: ", mfccmix.score_samples(X_test))

MFCCMIX

Train: 
Modelo:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [13]:
auc = calculate_aucroc(mfccmix, X_test, y_test)
auc

0.9150694444444444

## Treinar modelo GaussianMixture:

In [14]:
gmm = GaussianMixture()
gmm.fit(concat_X_train, y_train)


print("\n\nGMM")
print("\nTrain: ")
print("Modelo: ", gmm.predict(concat_X_train))
print("Real: ", y_train)
print("Score samples: ", gmm.score_samples(concat_X_train))

print("\nTest: ")
print("Modelo: ", gmm.predict(concat_X_test))
print("Real: ", y_test)
print("Score samples: ", gmm.score_samples(concat_X_test))



GMM

Train: 
Modelo:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 

In [15]:
auc = calculate_aucroc(gmm, concat_X_test, y_test)
auc

0.9096527777777776

## Treinar modelo OneClassSVM:

In [16]:
ocsvm = OneClassSVM(nu=0.05)
ocsvm.fit(concat_X_train, y_train)

print("\n\CVM")
print("\nTrain: ")
print("Modelo: ", ocsvm.predict(concat_X_train))
print("Real: ", y_train)
print("Score samples: ", ocsvm.score_samples(concat_X_train))

print("\nTest: ")
print("Modelo: ", ocsvm.predict(concat_X_test))
print("Real: ", y_test)
print("Score samples: ", ocsvm.score_samples(concat_X_test))


\CVM

Train: 
Modelo:  [ 1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1 -1  1  1  1 -1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1 -1  1 -1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1
  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1
  1  1  1  1  1 -1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1 -1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 -1  1  1  1  1  1  1  1  1

In [17]:
sub_y_test = y_test
for i in range(len(y_test)):
    if y_test[i] == 0:
        sub_y_test[i] = -1

sub_y_test

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1

In [18]:
auc = calculate_aucroc(ocsvm, concat_X_test, sub_y_test)
auc

0.5015972222222221

## Treinar modelo IsolationForest:

In [19]:
IsFo = IsolationForest()
IsFo.fit(concat_X_train, y_train)

print("\n\IsolationForest")
print("\nTrain: ")
print("Modelo: ", IsFo.predict(concat_X_train))
print("Real: ", y_train)
print("Score samples: ", IsFo.score_samples(concat_X_train))

print("\nTest: ")
print("Modelo: ", IsFo.predict(concat_X_test))
print("Real: ", y_test)
print("Score samples: ", IsFo.score_samples(concat_X_test))



\IsolationForest

Train: 
Modelo:  [-1  1  1  1 -1  1  1  1  1  1  1  1 -1  1  1  1  1 -1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1
  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1 -1 -1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1 -1  1 -1  1  1  1 -1
  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1
  1  1 -1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1
  1  1  1  1  1

In [20]:
sub_y_test = y_test
for i in range(len(y_test)):
    if y_test[i] == 0:
        sub_y_test[i] = -1

sub_y_test

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1

In [21]:
auc = calculate_aucroc(IsFo, concat_X_test, sub_y_test)
auc

0.9300694444444445

## Treinar modelo LocalOutlierFactor:

In [23]:
LOF = LocalOutlierFactor()

print("\n\LocalOutlierFactor")
print("\nTrain: ")
print("Modelo: ", LOF.fit_predict(concat_X_train))
print("Real: ", y_train)
print("Score samples: ", LOF.negative_outlier_factor_)

print("\nTest: ")
print("Modelo: ", LOF.fit_predict(concat_X_test))
print("Real: ", y_test)
print("Score samples: ", LOF.negative_outlier_factor_)



\LocalOutlierFactor

Train: 
Modelo:  [-1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1 -1  1  1  1  1 -1  1  1  1 -1 -1  1  1  1 -1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1
  1  1  1  1  1  1  1  1 -1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1
  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1 -1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1 -1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1
  1  1  1  1

In [25]:
sub_y_test = y_test
for i in range(len(y_test)):
    if y_test[i] == 0:
        sub_y_test[i] = -1

sub_y_test

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1

In [26]:
auc = calculate_aucroc(IsFo, concat_X_test, sub_y_test)
auc

0.9300694444444445

## shape

In [27]:
# Colocar todos os elementos do escore entre 0 e 1 e depois arredondar
def normalizar(x):
  rry = gmm.score_samples(x)

  mn = 1e18
  mx = -1e18
  for c in rry:
    mn = min(mn, c)
    mx = max(mx, c)

  save = []
  cont = 0
  for c in rry:
    if (mx-mn) == 0:
      save.insert(cont, 0)
    else:
      save.insert(cont, (c - mn)/(mx-mn))

    cont += 1

  return np.array(save)

print(normalizar(concat_X_train))

[0.13278945 0.85744879 0.85059019 0.89576331 0.19405267 0.93963519
 0.94752741 0.79540815 0.7033771  0.19872611 0.75068769 0.73036762
 0.37888058 0.73718506 0.83028504 0.84366589 0.56796713 0.1191677
 0.93134049 0.9095065  0.66084207 0.42434433 0.85584901 0.80356681
 0.23177369 0.89607694 0.89127507 0.68668001 0.86221451 0.78930867
 0.80735791 0.87074489 0.92818572 0.90463995 0.7149819  0.94499975
 0.80022971 0.7811567  0.5112586  0.78946185 0.8326029  0.91203516
 0.86668627 0.90023856 0.1389176  0.9684216  0.74704183 0.74558111
 0.87486789 0.13808343 0.47261177 0.76438175 0.70644822 0.22163227
 0.92897007 0.65097797 0.92931338 0.93670645 0.69340686 0.61081803
 0.77391662 0.69248333 0.68516653 0.78180763 0.81600968 0.82405927
 0.71928468 0.34316203 0.9804734  0.96800485 0.93105553 0.97552702
 0.75503154 0.48627604 0.87933124 0.6948319  0.89879256 0.64156839
 0.32476604 0.53354242 0.94156735 0.76289812 0.0712045  0.10935688
 0.85062022 0.80729993 0.82261487 0.77049621 0.94678495 0.89871

In [17]:
shap.initjs()

# Explicar as predições no conjunto de teste
explainer = shap.KernelExplainer(normalizar, concat_X_train)
shap_values = explainer.shap_values(concat_X_test)
shap.force_plot(explainer.expected_value, shap_values, concat_X_test)

Using 588 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.
  0%|          | 0/240 [00:00<?, ?it/s]

  1%|▏         | 3/240 [00:43<57:47, 14.63s/it]


KeyboardInterrupt: 